# Modeling

# Setup

In [ ]:
  def imports():
    from google.colab import auth
    from google.cloud import bigquery
    from google.colab import data_table
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import LabelEncoder
    from sklearn.model_selection import train_test_split
    from imblearn.under_sampling import RandomUnderSampler
    from sklearn.preprocessing import RobustScaler
    from sklearn.linear_model import LogisticRegression

In [ ]:
  #algorithm to fix imbalanced sample
  #!pip install -U imbalanced-learn

In [ ]:
# @title Setup
def setup():

  project = 'project-405913' # Project ID inserted based on the query results selected to explore
  location = 'EU' # Location inserted based on the query results selected to explore
  client = bigquery.Client(project=project, location=location)
  data_table.enable_dataframe_formatter()
  auth.authenticate_user()

  import pandas as pd

  # run query for feature building
  sql_query = ('''
  SELECT
  *
  FROM `project-405913.looker_tables.investments_vc_TOTAL_1_3_success`''')



  results_1 = client.query(sql_query).to_dataframe()#.transpose().reset_index(drop=True)
  results_1.shape
  return results_1

In [ ]:
df = setup()

## Setup for Decision Function

In [ ]:
def setup_decision_function():

  project = 'project-405913' # Project ID inserted based on the query results selected to explore
  location = 'EU' # Location inserted based on the query results selected to explore
  client = bigquery.Client(project=project, location=location)

# run query to load number of startups per round
  sql_query = ('''
  SELECT
  COUNTIF(round_A >0) as nb_round_A,
  COUNTIF(round_B >0) as nb_round_B,
  COUNTIF(round_C >0) as nb_round_C,
  COUNTIF(round_D >0) as nb_round_D,
  COUNTIF(round_E >0) as nb_round_E,
  COUNTIF(round_F >0) as nb_round_F,
  COUNTIF(round_G >0) as nb_round_G,
  COUNTIF(round_H >0) as nb_round_H,
  FROM `project-405913.looker_tables.investments_vc_TOTAL_1_3_success`''')

  df_nb_startups = client.query(sql_query).to_dataframe().transpose().reset_index(drop=True)
  df_nb_startups

  # run query to load average success rate per round
  sql_query = ('''
  SELECT
  AVG(CASE WHEN round_A >0 THEN success ELSE Null END) as avg_success_A,
  AVG(CASE WHEN round_B >0 THEN success ELSE Null END) as avg_success_B,
  AVG(CASE WHEN round_C >0 THEN success ELSE Null END) as avg_success_C,
  AVG(CASE WHEN round_D >0 THEN success ELSE Null END) as avg_success_D,
  AVG(CASE WHEN round_E >0 THEN success ELSE Null END) as avg_success_E,
  AVG(CASE WHEN round_F >0 THEN success ELSE Null END) as avg_success_F,
  AVG(CASE WHEN round_G >0 THEN success ELSE Null END) as avg_success_G,
  AVG(CASE WHEN round_H >0 THEN success ELSE Null END) as avg_success_H,
  AVG(success)

  FROM `project-405913.looker_tables.investments_vc_TOTAL_1_3_success`''')

  df_avg_success = client.query(sql_query).to_dataframe().transpose().reset_index(drop=True)

  # run query to load average funding amount per round
  sql_query = ('''
  SELECT
  AVG(CASE WHEN round_A >0 THEN round_A ELSE NULL END) as avg_funding_A,
  AVG(CASE WHEN round_B >0 THEN round_B ELSE NULL END) as avg_funding_B,
  AVG(CASE WHEN round_C >0 THEN round_C ELSE NULL END) as avg_funding_C,
  AVG(CASE WHEN round_D >0 THEN round_D ELSE NULL END) as avg_funding_D,
  AVG(CASE WHEN round_E >0 THEN round_E ELSE NULL END) as avg_funding_E,
  AVG(CASE WHEN round_F >0 THEN round_F ELSE NULL END) as avg_funding_F,
  AVG(CASE WHEN round_G >0 THEN round_G ELSE NULL END) as avg_funding_G,
  AVG(CASE WHEN round_H >0 THEN round_H ELSE NULL END) as avg_funding_H
  FROM `project-405913.looker_tables.investments_vc_TOTAL_1_3_success`''')

  df_avg_funding = client.query(sql_query).to_dataframe().transpose().reset_index(drop=True)

  # join into one dataframe
  indicators_table = df_nb_startups.merge(df_avg_success, left_index=True, right_index=True).merge(df_avg_funding, left_index=True, right_index=True)

  # renaming columns
  indicators_table = indicators_table.rename(columns={"0_x": "nb_records", "0_y":"avg_success", 0:"avg_funding"}, errors="raise")

  # add column with founding_round
  indicators_table.reset_index(inplace=True) # create column with row number
  indicators_table['index'] = indicators_table['index'].apply(lambda x: chr(x + 65)) # convert row number into letter
  indicators_table.rename(columns={"index":"funding_round"}, inplace=True)



  return indicators_table


In [ ]:
indicators_table = setup_decision_function()
indicators_table

,funding_round,nb_records,avg_success,avg_funding
0,A,9467,0.168797,6.820820e+06
1,B,5754,0.242092,1.354888e+07
2,C,2999,0.329777,2.100945e+07
3,D,1365,0.416117,2.857645e+07
4,E,545,0.491743,3.218808e+07
5,F,179,0.530726,4.981570e+07
6,G,36,0.638889,8.467007e+07
7,H,4,0.500000,1.759000e+08


# Creating features by cleaning and transforming data

### first and last funding at column feature

In [ ]:
def date_feature_creation(df):
  df.drop(index=36032, inplace = True)

  #changed columns to datetime
  df['first_funding_at'] = pd.to_datetime(df['first_funding_at'])
  df['founded_at'] = pd.to_datetime(df['founded_at'])
  df['last_funding_at'] = pd.to_datetime(df['last_funding_at'])

  #feature creation
  df['time_to_last_funding'] = df['last_funding_at'] - df['first_funding_at']
  df['time_to_last_funding']

  #fetching days from datetime
  df['time_to_last_funding'] = df['time_to_last_funding'].dt.days
  df['time_to_last_funding']

  df['time_to_first_funding'] = df['first_funding_at'] - df['founded_at']
  df['time_to_first_funding']
  df['time_to_first_funding'] = df['time_to_first_funding'].dt.days
  df['time_to_first_funding']

  df['time_to_first_funding'] = df['time_to_first_funding'].fillna(0)
  df['time_to_last_funding'] = df['time_to_last_funding'].fillna(0)

  return df

In [ ]:
df = date_feature_creation(df)
df

,name,category_list,nb_categories,market,funding_total_usd,status,country_code,state_code,region,city,...,round_D,round_E,round_F,round_G,round_H,name_clean,row_number,success,time_to_last_funding,time_to_first_funding
0,King World (Beijing) IT,|Video Games|,1,Video Games,5124450,operating,-,-,-,-,...,0,0,0,0,0,kingworld(beijing)it,1,0,670.0,0.0
1,Affectv,|Auctions|Video|Tablets|Mobile|Advertising|,5,Auctions,3030502,operating,-,-,-,-,...,0,0,0,0,0,affectv,1,0,0.0,932.0
2,TinyLoot,|Android|Mobile Games|Mobile|,3,Mobile Games,220000,operating,-,-,-,-,...,0,0,0,0,0,tinyloot,1,0,273.0,0.0
3,FanIQ,|Sports|,1,Sports,1008401,operating,-,-,-,-,...,0,0,0,0,0,faniq,1,0,979.0,608.0
4,NurseLiability.com,|Professional Services|Insurance|Legal|,3,Professional Services,0,operating,-,-,-,-,...,0,0,0,0,0,nurseliabilitycom,1,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50845,u-s-nursing-corporation,|Health and Wellness|,1,Health and Wellness,8000000,operating,USA,CO,Denver,Greenwood Village,...,0,0,0,0,0,usnursingcorporation,1,0,168.0,7583.0
50846,North Palm Beach County Surgery Center,|Health and Wellness|,1,Health and Wellness,192000,operating,USA,FL,Palm Beaches,Palm Beach Gardens,...,0,0,0,0,0,northpalmbeachcountysurgerycenter,1,0,0.0,0.0
50847,Centerphase Solutions,|Health and Wellness|,1,Health and Wellness,1900000,operating,USA,NJ,Newark,Upper Saddle River,...,0,0,0,0,0,centerphasesolutions,1,0,0.0,0.0
50848,docsink,|Health and Wellness|,1,Health and Wellness,1000000,operating,USA,NC,"Wilmington - Cape Fear, North Carolina",Wrightsville Beach,...,0,0,0,0,0,docsink,1,0,0.0,701.0


### Round Growth

In [ ]:
def rounds_features_creation(df):

  #df['round_B_growth'] = (df['round_B']/df['round_A'])
  df['round_B_growth'] = np.where((df['round_A'] != 0) & (df['round_B'] != 0), df['round_B'] / df['round_A'], 0)

  #df.round_B_growth.isna().sum()
  df.round_B_growth.unique()

  df['round_B_growth'] = pd.to_numeric(df['round_B_growth'], errors='coerce')
  df['round_B_growth'] = df['round_B_growth'].astype(float)

  df['round_C_growth'] = np.where(
      (df[['round_A', 'round_B']].sum(axis=1) != 0) & (df['round_C'] != 0),
      df['round_C'] / df[['round_A', 'round_B']].sum(axis=1),
      0)
  df['round_C_growth'].nunique()

  df['round_C_growth'] = pd.to_numeric(df['round_C_growth'], errors='coerce')
  df['round_C_growth'] = df['round_C_growth'].astype(float)

  df['round_D_growth'] = np.where(
    (df[['round_A', 'round_B', 'round_C']].sum(axis=1) != 0) & (df['round_D'] != 0),
    df['round_D'] / df[['round_A', 'round_B', 'round_C']].sum(axis=1),
    0)

  df.round_D_growth.nunique()

  df['round_D_growth'] = pd.to_numeric(df['round_D_growth'], errors='coerce')
  df['round_D_growth'] = df['round_D_growth'].astype(float)

  return df


In [ ]:
df = rounds_features_creation(df)

### Investment round volume

In [ ]:
def rounds_avg_features_creation(df):
  average_round_a = df['round_A'].mean()
  df['round_A_percentage'] = df['round_A'] / average_round_a

  average_round_b = df['round_B'].mean()
  df['round_B_percentage'] = df['round_B'] / average_round_b

  average_round_c = df['round_C'].mean()
  df['round_C_percentage'] = df['round_C'] / average_round_c

  average_round_d = df['round_D'].mean()
  df['round_D_percentage'] = df['round_D'] / average_round_d

  return df

In [ ]:
df = rounds_avg_features_creation(df)

# Building the Model

In [ ]:
def model_build(df):
  label_encoder_market = LabelEncoder()

  #label encoder to convert string to int64
  df['market_numeric'] = label_encoder_market.fit_transform(df['market'])

  label_encoder_country = LabelEncoder()
  df['country_numeric'] = label_encoder_country.fit_transform(df['country_code'])

  #loading the target and features
  y = df['success']
  X = df[['market_numeric','country_numeric','nb_categories','round_B_growth',
       'round_C_growth', 'round_D_growth', 'round_A_percentage',
       'round_B_percentage', 'round_C_percentage', 'round_D_percentage','time_to_last_funding', 'time_to_first_funding']]

  #train-test split
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

  # balancing the 0 and 1 to avoid bias

  rus = RandomUnderSampler(random_state=42)
  X_train_res, y_train_res = rus.fit_resample(X_train, y_train)

  y_train_res = y_train_res
  y_test = pd.DataFrame(y_test)

  scaler = RobustScaler()
  X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_res), columns=scaler.get_feature_names_out(), index=X_train_res.index)
  X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=scaler.get_feature_names_out(),index=X_test.index)

  # train model
  clf = LogisticRegression()
  clf.fit(X_train_scaled, y_train_res)

  # store predictions
  y_pred = pd.DataFrame(clf.predict(X_test_scaled))

  #print(y_pred.value_counts())

  accuracy_test = clf.score(X_test_scaled, y_test)

  #print(f"test data accuracy = {round(accuracy_test,3)}")

  return clf

In [ ]:
model = model_build(df)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Building Function to generate dataset from Input

In [ ]:
# define a function to create a dataframe for the prediction

# define function and the inputs the user has to give
def create_dataset(market,country,nb_categories,r_A,r_B,r_C,r_D,found_date,first_fund_date,last_fund_date):

  # encode string values
  market_num = label_encoder_market.transform([market])
  country_num = label_encoder_country.transform([country])

  # Date-Features
  # do the necesary conversions and calculations to create the feature values
  if first_fund_date != "0":
    first_funding_at = pd.to_datetime(first_fund_date)
    time_to_first_funding = "dummy" # we need to assign a dummy value so that the function doesn't break
  else: time_to_first_funding = 0

  if last_fund_date != "0":
    last_funding_at = pd.to_datetime(last_fund_date)
    time_to_last_funding = "dummy"
  else: time_to_last_funding = 0

  if found_date != "0":
    founded_at = pd.to_datetime(found_date)
  else: return "go get a company that actually exists, Dude! "

  #Date-Calculations
  if time_to_last_funding != 0:
    time_to_last_funding = last_funding_at - first_funding_at
    time_to_last_funding = time_to_last_funding.days
  if time_to_first_funding != 0:
    time_to_first_funding = first_funding_at - founded_at
    time_to_first_funding = time_to_first_funding.days

  # Round Growth Features
  if r_A != 0 & r_B != 0:
    round_B_growth = r_B / r_A
  else: round_B_growth = 0

  if (r_A + r_B) != 0 & r_C != 0:
    round_C_growth = r_C / (r_A + r_B)
  else: round_C_growth = 0

  if (r_A + r_B + r_C) & r_D != 0:
    round_D_growth = r_D / (r_A + r_B + r_C)
  else: round_D_growth = 0

  # Round Size Features
  if r_A != 0:
    round_A_percentage = r_A / average_round_a
  else: round_A_percentage = 0

  if r_B != 0:
    round_B_percentage = r_B / average_round_b
  else: round_B_percentage = 0

  if r_C != 0:
    round_C_percentage = r_C / average_round_c
  else: round_C_percentage = 0

  if r_D != 0:
    round_D_percentage = r_D / average_round_d
  else: round_D_percentage = 0

  # build a data array to create a dataframe
  data = [[market_num,country_num,nb_categories,round_B_growth,round_C_growth, round_D_growth, round_A_percentage,round_B_percentage, round_C_percentage, round_D_percentage,time_to_last_funding, time_to_first_funding]]

  # build a dataframe, which can then be used for prediction
  startup_company = pd.DataFrame(data, columns=['market_numeric','country_numeric','nb_categories','round_B_growth',
      'round_C_growth', 'round_D_growth', 'round_A_percentage', 'round_B_percentage', 'round_C_percentage', 'round_D_percentage','time_to_last_funding', 'time_to_first_funding'])

  # scale the data, using the same fitted scaler from the Model
  startup_company_scaled = pd.DataFrame(scaler.transform(startup_company), columns=scaler.get_feature_names_out())

  return startup_company_scaled

# Building Function as decision assistant

In [ ]:
# The function goes into indicators_table, collects average success and funding size
# and then calculates the funding we'd be willing to pay, based on the predicted success rate

def to_fund_or_not(requested_round, required_funding, predicted_success):
        row = indicators_table[indicators_table['funding_round'] == requested_round] # pick the row of the requested funding round
        if row.empty:
            return "Model does not cover the requested round. Please select from A to H."
        avg_funding = row["avg_funding"].values[0]
        avg_success = row["avg_success"].values[0]

        # just for robustness, check it's not empty
        if pd.isnull(avg_funding) or pd.isnull(avg_success):
            return "Missing data on average funding for this round type."

        max_funding = avg_funding * predicted_success / avg_success
        if required_funding <= max_funding:
            return print(f"Positive - The funding request of", required_funding, "is below the maximum recommendation of",round(max_funding, 0), "by", round((max_funding*100/required_funding-100),1) ,"%")
        return print(f"Negative - The funding request of", required_funding, "is above the maximum recommendation of",round(max_funding, 0), "by", round((max_funding*100/required_funding-100),1)*-1 ,"%")


# Prediction function

In [ ]:
def total_function(clf):

  # gather inputs
  requested_round = input("What round are you raising? A, B, C or D?")
  required_funding = int(input("How much money are your raising?"))
  market = input("What's the key market?")
  country = input("What's the HQ country code?")
  nb_categories= int(input("how many categories are you serving?"))
  r_A = int(input("What was the value of your Round A, if any?"))
  r_B = int(input("What was the value of your Round B, if any?"))
  r_C = int(input("What was the value of your Round C, if any?"))
  r_D = int(input("What was the value of your Round D, if any?"))
  found_date = input("When was your founding date in YYYY-MM-DD?")
  first_fund_date = input("When was your first funding round date in YYYY-MM-DD?")
  last_fund_date = input("When was your last funding round date in YYYY-MM-DD?")

  # create a dataset
  startup = create_dataset(market,country,nb_categories,r_A,r_B,r_C,r_D,found_date,first_fund_date,last_fund_date)

  # do the prediction
  y_pred = clf.predict(startup)[0]
  if y_pred == 1.0:
    print(f"Success - The Model predicts this venture to be successful! The confidence for this prediction is {round(clf.predict_proba(startup)[0][1],2)}")

  else: print(f"Failure - The Model predicts this venture not to be successful! The confidence for this prediction is {round(clf.predict_proba(startup)[0][0],2)}")

  # do the decision
  to_fund_or_not(requested_round, required_funding, round(clf.predict_proba(startup)[0][0],2))

In [ ]:
total_function(model)

KeyboardInterrupt: ignored